<a href="https://colab.research.google.com/github/Matrixmax/Dive_into_DeepLearning/blob/main/0x17_%E6%A8%A1%E5%9E%8B%E5%8F%82%E6%95%B0%E7%9A%84%E8%AE%BF%E9%97%AE%EF%BC%8C%E5%88%9D%E5%A7%8B%E5%8C%96%E5%92%8C%E5%85%B1%E4%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

在3.3节（线性回归的简洁实现）中，我们通过init模块来初始化模型的参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。与之前不同的是，在这里我们从nn中导入了init模块，它包含了多种模型初始化方法。

In [ ]:
import torch
from torch import nn
from torch.nn import init

# pytorch已经进行默认初始化
net = nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))

print(net)
X = torch.rand(2,4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 1. 访问模型参数
回忆一下上一节中提到的Sequential类与Module类的继承关系。对于Sequential实例中含模型参数的层，我们可以通过Module类的parameters()或者named_parameters()方法来访问所有参数（以迭代器的形式返回），后者除了返回参数Tensor外还会返回其名字。下面，访问多层感知机net的所有参数：

In [ ]:
print(type(net.named_parameters()))
for name,param in net.named_parameters():
    print(str(name)+" : "+str(param.size()))

<class 'generator'>
0.weight : torch.Size([3, 4])
0.bias : torch.Size([3])
2.weight : torch.Size([1, 3])
2.bias : torch.Size([1])


可见返回的名字自动加上了层数的索引作为前缀。 我们再来访问net中单层的参数。对于使用Sequential类构造的神经网络，我们可以通过方括号[]来访问网络的任一层。索引0表示隐藏层为Sequential实例最先添加的层。

In [ ]:
for name,param in net[0].named_parameters():
    print(name,param.size(),type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


因为这里是单层的所以没有了层数索引的前缀。另外返回的param的类型为torch.nn.parameter.Parameter，其实这是Tensor的子类，和Tensor不同的是如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里，来看下面这个例子。

In [ ]:
class MyModule(nn.Module):
    def __init__(self,**kwargs):
        super(MyModule,self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20,20))
        self.weight2 = torch.rand(20,20)
    def forward(self,x):
        pass

n = MyModule()
for name,param in n.named_parameters():
    print(name)

weight1


上面的代码中weight1在参数列表中但是weight2却没在参数列表中。

因为Parameter是Tensor，即Tensor拥有的属性它都有，比如可以根据data来访问参数数值，用grad来访问参数梯度。

In [ ]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播之前，梯度为None
Y.backward()
print(weight_0.grad)


tensor([[-0.0427, -0.4227,  0.4240,  0.0940],
        [ 0.4434,  0.3806,  0.4065,  0.2683],
        [ 0.3830,  0.3362,  0.3729, -0.0349]])
None
tensor([[-0.1669, -0.5457, -0.5394, -0.8498],
        [ 0.1356,  0.4435,  0.4384,  0.6907],
        [-0.0597, -0.1952, -0.1930, -0.3040]])


## 2.初始化模型参数
我们经常需要使用其他方法来初始化权重。PyTorch的init模块里提供了多种预设的初始化方法。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [ ]:
for name,param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param,mean=0,std=0.01)
        print(name,param.data)

0.weight tensor([[ 0.0088, -0.0079, -0.0070, -0.0157],
        [ 0.0031, -0.0045,  0.0131,  0.0041],
        [ 0.0027, -0.0098,  0.0134,  0.0007]])
2.weight tensor([[-0.0054, -0.0123,  0.0017]])


In [ ]:
# 下面使用常数来初始化权重参数
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 3.自定义初始化参数
有时候我们需要的初始化方法并没有在init模块中提供。这时，可以实现一个初始化方法，从而能够像使用其他初始化方法那样使用它。在这之前我们先来看看PyTorch是怎么实现这些初始化方法的，例如torch.nn.init.normal_：

In [ ]:
def normal_(tensor,mean=0,std=1):
    with ttorch.no_grad():
        return tensor.normal_(mean,std)

可以看到这就是一个inplace改变Tensor值的函数，而且这个过程是不记录梯度的。 类似的我们来实现一个自定义的初始化方法。在下面的例子里，我们令权重有一半概率初始化为0，有另一半概率初始化为[−10,−5][−10,−5]和[5,10][5,10]两个区间里均匀分布的随机数。

In [10]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor *= (tensor.abs()>=5).float()

for name,param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name,param.data)

0.weight tensor([[-0.0000,  5.3147, -0.0000,  8.7943],
        [ 7.7569, -7.8153,  9.3808,  6.1814],
        [-0.0000, -5.5513,  0.0000,  7.8947]])
2.weight tensor([[0., 0., -0.]])


我们还可以通过改变这些参数的data来改写模型参数值同时不会影响梯度:

In [12]:
for name,param in net.named_parameters():
    if 'bias' in name:
        param.data +=1
        print(name,param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 4.共享模型参数
在有些情况下，我们希望在多个层之间共享模型参数。4.1.3节提到了如何共享模型参数: Module类的forward函数里多次调用同一个层。此外，如果我们传入Sequential的模块是同一个Module实例的话参数也是共享的，下面来看一个例子:

In [14]:
linear = nn.Linear(1,1,bias=False)
net = nn.Sequential(linear,linear)
print(net)
for name,param in net.named_parameters():
    init.constant_(param,val=3)
    print(name,param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


在内存中，这两个线性层其实一个对象:

In [15]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:



In [16]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) # 单次梯度是3，两次所以就是6


tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
